In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('weather_disaster_dataset.csv')

In [4]:
thresholds = {
    'Temperature_C': {'low': -10, 'high': 40},   # Dangerous temp below -10 or above 40°C
    'Humidity_%': {'low': 10, 'high': 85},       # Dangerous humidity above 85%
    'Rain_mm': {'low': 0, 'high': 100},          # Dangerous rain above 100 mm
    'Wind_Speed_kmph': {'low': 0, 'high': 70},   # Dangerous wind speed above 70 km/h
    'Pressure_hPa': {'low': 980, 'high': 1050},  # Dangerous pressure below 980 hPa (storm)
    'Visibility_km': {'low': 1, 'high': 20},     # Dangerous visibility below 1 km
    'Cloud_Cover_%': {'low': 0, 'high': 100}     # High cloud cover (informational)
}

In [5]:
def calculate_risk(row):
    risk_score = 0
    danger = []
    
    # Check each parameter against its threshold
    for col, limits in thresholds.items():
        if col in row:
            if row[col] < limits['low'] or row[col] > limits['high']:
                risk_score += 1
                danger.append(col)
    
    return risk_score, danger

In [6]:
df['Risk_Score'], df['Danger_Parameters'] = zip(*df.apply(calculate_risk, axis=1))

In [7]:
def alert_level(row):
    if row['Risk_Score'] >= 4:
        return 'Red Alert'
    elif row['Risk_Score'] == 3:
        return 'Orange Alert'
    elif row['Rain_mm'] > 100:  # Specific high risk for rain
        return 'High Rain Risk'
    elif row['Risk_Score'] > 0:
        return 'Yellow Alert'
    else:
        return 'Normal'

In [8]:
df['Alert_Level'] = df.apply(alert_level, axis=1)

In [9]:
print(df[['Date', 'Disaster_Type', 'Temperature_C', 'Rain_mm', 'Risk_Score', 'Alert_Level']].head(10))

         Date Disaster_Type  Temperature_C  Rain_mm  Risk_Score  \
0  2023-07-24     Hurricane           22.1     12.0           0   
1  2023-03-08       Tornado           20.2    123.8           1   
2  2023-04-13      Wildfire           20.5      5.6           1   
3  2023-08-27    Earthquake           15.7      0.7           0   
4  2023-09-03      Wildfire           28.7     16.1           0   
5  2023-11-17     Hurricane           17.5      4.7           0   
6  2023-09-06       Tornado           32.9     19.1           1   
7  2023-11-01         Flood           22.5     19.9           0   
8  2023-02-21         Flood           15.6     83.9           0   
9  2023-04-03       Tornado           14.5    348.4           2   

      Alert_Level  
0          Normal  
1  High Rain Risk  
2    Yellow Alert  
3          Normal  
4          Normal  
5          Normal  
6    Yellow Alert  
7          Normal  
8          Normal  
9  High Rain Risk  


In [10]:
df.to_csv('updated_weather_disaster_dataset_with_alerts.csv', index=False)

In [11]:
red_alerts = df[df['Alert_Level'] == 'Red Alert']
orange_alerts = df[df['Alert_Level'] == 'Orange Alert']
yellow_alerts = df[df['Alert_Level'] == 'Yellow Alert']

In [12]:
print(f"Red Alerts: {len(red_alerts)}")
print(f"Orange Alerts: {len(orange_alerts)}")
print(f"Yellow Alerts: {len(yellow_alerts)}")

Red Alerts: 0
Orange Alerts: 5
Yellow Alerts: 67
